# RecallrAI Python SDK Examples

This notebook demonstrates the usage of the RecallrAI Python SDK, a system that enables AI assistants to form meaningful connections between conversations, just like human memory.

## Installation
To install the RecallrAI Python SDK, use the following command:

In [ ]:
%pip install recallrai python-dotenv --upgrade --quiet


In [ ]:
import recallrai
print(recallrai.__version__)


## Initialization

First, create a client instance with your API key and project ID:

In [ ]:
import os
from recallrai import RecallrAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Replace with your actual API key and project ID
api_key = os.getenv("RECALLRAI_API_KEY")
project_id = os.getenv("RECALLRAI_PROJECT_ID")
base_url = os.getenv("RECALLRAI_BASE_URL", "https://api.recallrai.com")
timeout = int(os.getenv("RECALLRAI_TIMEOUT", "60"))

if not api_key or not project_id:
    raise ValueError("Please set RECALLRAI_API_KEY and RECALLRAI_PROJECT_ID in your .env file.")

client = RecallrAI(
    api_key=api_key,
    project_id=project_id,
    base_url=base_url,  # custom endpoint if applicable
    timeout=timeout,    # seconds
)


## User Management

### Create a User

In [ ]:
from recallrai.exceptions import UserAlreadyExistsError
try:
    user = client.create_user(user_id="user123", metadata={"name": "John Doe"})
    print(f"Created user: {user.user_id}")
    print(f"User metadata: {user.metadata}")
    print(f"Created at: {user.created_at}")
except UserAlreadyExistsError as e:
    print(f"Error: {e}")


### Get a User

In [ ]:
from recallrai.exceptions import UserNotFoundError
try:
    user = client.get_user("user123")
    print(f"User metadata: {user.metadata}")
    print(f"Last active: {user.last_active_at}")
except UserNotFoundError as e:
    print(f"Error: {e}")


### List Users

In [ ]:
user_list = client.list_users(offset=0, limit=10, metadata_filter={"role": "admin"})
print(f"Total users: {user_list.total}")
print(f"Has more users: {user_list.has_more}")

for user in user_list.users:
    print(f"User ID: {user.user_id}")
    print(f"Metadata: {user.metadata}")
    print(f"Created at: {user.created_at}")
    print(f"Last active: {user.last_active_at}")
    print("---")


### Update a User

In [ ]:
from recallrai.exceptions import UserNotFoundError, UserAlreadyExistsError
try:
    user = client.get_user("user123")
    updated_user = user.update(
        new_metadata={"name": "John Doe", "role": "admin"},
        new_user_id="john_doe"
    )
    print(f"Updated user ID: {updated_user.user_id}")
    print(f"Updated metadata: {updated_user.metadata}")
except UserNotFoundError as e:
    print(f"Error: {e}")
except UserAlreadyExistsError as e:
    print(f"Error: {e}")


### Delete a User

In [ ]:
from recallrai.exceptions import UserNotFoundError
try:
    user = client.get_user("john_doe")
    user.delete()
    print("User deleted successfully")
except UserNotFoundError as e:
    print(f"Error: {e}")


## Session Management

### Create a Session

In [ ]:
from recallrai.exceptions import UserNotFoundError
from recallrai.session import Session

try:
    # First, get the user
    user = client.get_user("user123")
    
    # Create a session for the user.
    session: Session = user.create_session(
        auto_process_after_seconds=600,
        metadata={"type": "chat"}
    )
    print("Created session id:", session.session_id)
except UserNotFoundError as e:
    print(f"Error: {e}")


### Get an Existing Session

In [ ]:
from recallrai.exceptions import UserNotFoundError, SessionNotFoundError

try:
    # First, get the user
    user = client.get_user("user123")
    
    # Retrieve an existing session by its ID
    session = user.get_session(session_id="session-uuid")
    print("Session status:", session.get_status())
except UserNotFoundError as e:
    print(f"Error: {e}")
except SessionNotFoundError as e:
    print(f"Error: {e}")


### List Sessions

In [ ]:
from recallrai.exceptions import UserNotFoundError

try:
    # First, get the user
    user = client.get_user("user123")
    
    # List sessions for this user with optional metadata filters
    session_list = user.list_sessions(
        offset=0,
        limit=10,
        metadata_filter={"type": "chat"},           # optional
        user_metadata_filter={"role": "admin"}       # optional
)
    print(f"Total sessions: {session_list.total}")
    print(f"Has more sessions: {session_list.has_more}")
    for sess in session_list.sessions:
        print(sess.session_id, sess.status)
except UserNotFoundError as e:
    print(f"Error: {e}")


### Session – Process Session

In [ ]:
from recallrai.exceptions import UserNotFoundError, SessionNotFoundError, InvalidSessionStateError

try:
    session.process()
except UserNotFoundError as e:
    print(f"Error: {e}")
except SessionNotFoundError as e:
    print(f"Error: {e}")
except InvalidSessionStateError as e:
    print(f"Error: {e}")


### Session – Getting Status

In [ ]:
from recallrai.exceptions import UserNotFoundError, SessionNotFoundError

try:
    status = session.get_status()
    print("Session status:", status)
except UserNotFoundError as e:
    print(f"Error: {e}")
except SessionNotFoundError as e:
    print(f"Error: {e}")


### Session – List Messages

In [ ]:
from recallrai.exceptions import UserNotFoundError, SessionNotFoundError

try:
    messages = session.get_messages()
    for msg in messages:
        print(f"{msg.role.value.capitalize()} (at {msg.timestamp}): {msg.content}")
except UserNotFoundError as e:
    print(f"Error: {e}")
except SessionNotFoundError as e:
    print(f"Error: {e}")


## User Memories

### List User Memories (with optional category filters)

In [ ]:
from recallrai.exceptions import UserNotFoundError

try:
    user = client.get_user("user123")
    memories = user.list_memories(
        categories=["food_preferences", "allergies"],  # optional
        offset=0,
        limit=20,
    )
    for mem in memories.items:
        print(f"Memory ID: {mem.memory_id}")
        print(f"Categories: {mem.categories}")
        print(f"Content: {mem.content}")
        print(f"Created at: {mem.created_at}")
        print("---")
    print(f"Has more?: {memories.has_more}")
    print(f"Total memories: {memories.total}")
except UserNotFoundError as e:
    print(f"Error: {e}")
